In [1]:
import os, glob
import xarray as xr
import numpy as np
import os, glob, subprocess

In [2]:
    def get_mesh_latlon_range(path_CTSM_case):
        cwd = os.getcwd()
        os.chdir(path_CTSM_case)
        out = subprocess.run('./xmlquery LND_DOMAIN_MESH', shell=True, capture_output=True)
        infileMESH = out.stdout.decode().strip().split(':')[1].strip()
        os.chdir(cwd)

        # find coordinates of basins with mask == 1
        ds_basin = xr.load_dataset(infileMESH)
        nodeCoords = ds_basin.nodeCoords.values
        elementConn = ds_basin.elementConn.values
        numElementConn = ds_basin.numElementConn.values
        nodeCoords_valid = np.zeros([0, 2])

        for i in range(ds_basin.elementCount.size):
            if ds_basin.elementMask.values[i] == 1:
                index_elementCount = [i]
                index_connectionCount = np.arange(np.sum(numElementConn[:i]), np.sum(numElementConn[:i+1])).astype(int)
                index_nodeCount = elementConn[index_connectionCount].astype(int) - 1 # index starts from 0
                nodeCoords_valid = np.vstack((nodeCoords_valid, nodeCoords[index_nodeCount, :]))
        
        buffer = 0
        lat_range = [np.min(nodeCoords_valid[:, 1])-buffer, np.max(nodeCoords_valid[:, 1])+buffer]
        lon_range = [np.min(nodeCoords_valid[:, 0])-buffer, np.max(nodeCoords_valid[:, 0])+buffer]

        return lat_range, lon_range

In [6]:

folders = glob.glob(f'/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/*_SubsetForcing')
#folders = [f'/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_0_SubsetForcing']
folders.sort()
print('Folder number:', len(folders))

num = 0
rmfolders = []
for f in folders:
    
    # forcing range
    files = glob.glob(f'{f}/Solar/subset_clmforc.E5LEME.c2023.010x010.Solar.1*.nc')[0]
    ds = xr.open_dataset(files)
    
    lonr = [ds.LONGXY.values.min()-0.05, ds.LONGXY.values.max()+0.05] # 0.05 to grid boundary
    latr = [ds.LATIXY.values.min()-0.05, ds.LATIXY.values.max()+0.05] # 0.05 to grid boundary
    
    # mesh range
    folder = f.replace('_SubsetForcing', '')
    latm, lonm = get_mesh_latlon_range(folder)
    print(folder)
    print('mesh/forcing', latm, latr)
    print('mesh/forcing', lonm, lonr)
    if latm[0]<latr[0] or lonm[0]<lonr[0] or latm[1]>latr[1] or lonm[1]>lonr[1]:
        print(folder)
        print('mesh/forcing', latm, latr)
        print('mesh/forcing', lonm, lonr)
        
        num = num + 1
        # os.system(f'rm -r {f}')
        # f = f.replace('_SubsetForcing', '')
        # os.system(f'rm -r {f}')
        rmfolders.append(f)

Folder number: 73
/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_0
mesh/forcing [46.683682, 47.303017] [46.49999923706055, 47.50000076293945]
mesh/forcing [291.035932, 291.92393] [290.90001220703124, 292.09998779296876]
/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_10
mesh/forcing [43.469593, 43.646992] [43.2999984741211, 43.8]
mesh/forcing [287.981155, 288.260551] [287.8000061035156, 288.40000610351564]
/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_11
mesh/forcing [41.477083, 41.512278] [41.2999984741211, 41.7000015258789]
mesh/forcing [288.137389, 288.180278] [287.99998779296874, 288.3]
/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_12
mesh/forcing [41.823944, 41.9705] [41.7, 42.099999237060544]
mesh/forcing [287.773278, 287.887944] [287.59999389648436, 288.00001220703126]
/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_13
mesh/forcing [41.668833, 41.854056] [